# Mount to Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Improt required libraries

In [8]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os
import re
from matplotlib import pyplot as plt
import scipy.io as sio
import numpy as np
import pickle as pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from scipy.signal import spectrogram
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from scipy import signal
import scipy.io
import os
import math

import tensorflow as tf
import keras
from tensorflow.keras.utils import to_categorical
# from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, LSTM, Conv2D, MaxPooling2D, RepeatVector
import warnings
warnings.filterwarnings('ignore')

# Data Pre-processing

In [4]:
dataset_path = '/content/drive/My Drive/Major/BED_dataset_RAW_PARSED/'

In [5]:
def add_gaussian_noise(eeg_data, proportion=0.01, mean=0, std=1):
    # Calculate the standard deviation of the noise
    noise_std = proportion * np.std(eeg_data)

    # Generate Gaussian noise with the same shape as the EEG data
    noise = np.random.normal(mean, noise_std, eeg_data.shape)

    # Add the noise to the EEG data
    noisy_eeg_data = eeg_data + noise

    return noisy_eeg_data

In [9]:
def load_eeg_data(directory):
    data = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith('.mat'):
            filepath = os.path.join(directory, filename)

            mat_contents = scipy.io.loadmat(filepath)
            recordings = mat_contents['recording']

            recordings = recordings[:30208, 2:-1]
            recordings = recordings.T

            data.append(recordings)

            # Extract label from filename
            start_index = filename.index('s') + 1
            end_index = filename.index('_')
            # Extract the substring between 's' and '_'
            number = filename[start_index:end_index]
            person_id = int(number) # Convert the extracted number to an integer

            label = person_id - 1
            labels.append(label)

            # Augmentation - Gussian Noise
            noisy_data = add_gaussian_noise(recordings);
            data.append(noisy_data)
            labels.append(label)

    data = np.array(data)
    labels = np.array(labels)

    return data, labels

In [ ]:
eeg_data, labels = load_eeg_data(dataset_path)

In [11]:
eeg_Label = []
for i in labels:
    for j in range (0,14):
        eeg_Label.append(i)

In [ ]:
y = to_categorical(eeg_Label)

In [13]:
import functools
def combine_dims(a, i=0, n=1):
  s = list(a.shape)
  combined = functools.reduce(lambda x,y: x*y, s[i:i+n+1])
  return np.reshape(a, s[:i] + [combined] + s[i+n+1:])

In [ ]:
eeg_Data = combine_dims(np.array(eeg_data), 0)

# Feature Extraction

In [ ]:
fs = 256

size_dataset = len(eeg_Data)
nperseg = 512
noverlap = 256
f_size = math.ceil(((nperseg + 1)/2))
t_size = int((eeg_Data[0].size - noverlap ) / (nperseg - noverlap))
X_full = np.ndarray(shape=(size_dataset, f_size, t_size, 3))

for i in range(0, size_dataset):
  for j in range(0,3):
    X = eeg_Data[i]
    f, t, Sxx = spectrogram(X, fs, nperseg = nperseg, noverlap = noverlap, mode='psd')
    X_full[i, :, :, j] = Sxx

X_full /= 255

# Load Models

In [17]:
models_path = '/content/drive/My Drive/saved_models'

model_files = [filename for filename in os.listdir(models_path) if filename.endswith('.h5')]

loaded_models = []

for filename in model_files:
    # Extract channel name from filename
    model_path = os.path.join(models_path, filename)
    model = load_model(model_path)
    loaded_models.append(model)
    print("%s loaded successfully", filename)

print("\n All Models loaded successfully!!")


BED_F3.h5 loaded successfully
BED_FC5.h5 loaded successfully
BED_AF3.h5 loaded successfully
BED_F7.h5 loaded successfully
BED_T7.h5 loaded successfully
BED_P7.h5 loaded successfully
BED_O1.h5 loaded successfully
BED_O2.h5 loaded successfully
BED_P4.h5 loaded successfully
BED_T8.h5 loaded successfully
BED_F8.h5 loaded successfully
BED_AF4.h5 loaded successfully
BED_FC6.h5 loaded successfully
BED_F4.h5 loaded successfully

 All Models loaded successfully!!


# Get predictions

In [20]:
pred_y_full = []

for model in loaded_models:
    pred_y = model.predict(X_full)
    pred_y = np.argmax(pred_y,axis=1)

    pred_y_full.append(pred_y)

pred_y_full = np.array(pred_y_full)

pred_y_full.shape

56/56 [==============================] - 27s 474ms/step


(14, 1764)

In [21]:
# convert categorial labels to initial label
label_y = np.argmax(y, axis=1)

# Voting

In [22]:
import numpy as np

def hard_voting(predictions):
    # Calculate the majority class label
    majority_label = np.bincount(predictions).argmax()

    # Count the occurrences of the majority label
    majority_count = np.count_nonzero(predictions == majority_label)

    # Check if the majority label appears in at least 75% of the predictions
    if majority_count >= 0.75 * len(predictions):
        # Return the majority label as the final prediction
        return majority_label
    else:
        # Return -1 if 75% of the elements are not the same
        return -1


In [23]:
voted_Y = []
for j in range(len(label_y)):
  result = []
  for i in range(0,14):
    result.append(pred_y_full[i][j])

  result = np.array(result)
  pred = hard_voting(result)
  voted_Y.append(pred)

voted_Y = np.array(voted_Y)

# Plot metrics

In [27]:
print(metrics.classification_report(label_y,voted_Y))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        84
           1       0.95      1.00      0.98        84
           2       1.00      0.98      0.99        84
           3       1.00      1.00      1.00        84
           4       1.00      1.00      1.00        84
           5       1.00      0.98      0.99        84
           6       1.00      1.00      1.00        84
           7       1.00      0.98      0.99        84
           8       1.00      0.98      0.99        84
           9       1.00      0.98      0.99        84
          10       0.98      1.00      0.99        84
          11       1.00      1.00      1.00        84
          12       0.95      1.00      0.98        84
          13       1.00      0.98      0.99        84
          14       0.98      0.98      0.98        84
          15       1.00      1.00      1.00        84
          16       1.00      1.00      1.00        84
          17       1.00    